### Étude de cas sur un dataset de la restauration

L'étude se fait sur le fichier resto.csv, un dataset représentant des commandes de la chaîne de restauration Chipotle.

In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df1 = pd.read_csv("source/resto.csv")

In [60]:
df1

,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price
0,0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,1,Izze,[Clementine],$3.39
2,2,1,1,Nantucket Nectar,[Apple],$3.39
3,3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
...,...,...,...,...,...,...
4617,4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75
4618,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75
4619,4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25
4620,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75


##### Commentaires sur cet ensemble de données

- Peu de colonnes à analyser
- _Unnamed: 0_ est l'id de la ligne de donnée, cela représente un article
- _order\_id_ est l'id d'une commande; une commande peut donc comprendre plusieurs articles
- _item\_name_ est le nom de l'article
- _choice\_description_ contient un détail de l'article commandé lorsque celui-ci n'est pas assez explicite (exemple: on commande un jus, mais un jus de quoi ?) ou lorsque celui contient plusieurs ingrédients à choisir (exemple: on commande un burrito, mais on prend quelle sauce, surplus fromage, etc... ?)
- _item\_price_ est le prix de l'article

A priori, les 2 principales complexités de cet ensemble de données sont la colonne _choice\_description_ qui contient des valeurs qui ne sont pas atomiques, et le fait de pouvoir grouper le prix pour chaque commande plutôt que pour chaque article.

In [61]:
df1.describe()

,Unnamed: 0,order_id,quantity
count,4622.000000,4622.000000,4622.000000
mean,2310.500000,927.254868,1.075725
std,1334.400802,528.890796,0.410186
min,0.000000,1.000000,1.000000
25%,1155.250000,477.250000,1.000000
50%,2310.500000,926.000000,1.000000
75%,3465.750000,1393.000000,1.000000
max,4621.000000,1834.000000,15.000000


In [62]:
df1.dtypes

Unnamed: 0             int64
order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

La description du DataFrame ne donne pour l'instant que peu d'informations utiles. C'est assez logique, 3 des colonnes ne sont pas quantifiables en l'état.\
Pourtant, item\_price est une variable quantifiable. Si tous les prix sont en dollars, alors on pourra retirer les $ et transformer ces valeurs en float afin de pouvoir mesurer cette variable.

In [63]:
df1["item_price"].str.contains("^\$").value_counts()

True    4622
Name: item_price, dtype: int64

Les 4622 données de prix sont toutes en dollars et commencent par le $. On peut donc traiter cette colonne sans risque de perdre des données et la transformer en float.

In [64]:
# On supprime le dollar à l'aide d'une regex
df1["item_price"] = df1["item_price"].replace(regex=r"\$", value="")
df1

,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price
0,0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,1,Izze,[Clementine],3.39
2,2,1,1,Nantucket Nectar,[Apple],3.39
3,3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
...,...,...,...,...,...,...
4617,4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75
4618,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75
4619,4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25
4620,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75


In [65]:
df1.dtypes

Unnamed: 0             int64
order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

In [66]:
# On change le type de la colonne item_price en float
df1["item_price"] = df1["item_price"].astype(float)

In [67]:
df1.dtypes

Unnamed: 0              int64
order_id                int64
quantity                int64
item_name              object
choice_description     object
item_price            float64
dtype: object

In [68]:
df1["item_price_unit"] = df1["item_price"]/df1["quantity"]

In [69]:
df1

,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price,item_price_unit
0,0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,2.39
1,1,1,1,Izze,[Clementine],3.39,3.39
2,2,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,2.39
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,8.49
...,...,...,...,...,...,...,...
4617,4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75,11.75
4618,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75,11.75
4619,4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25,11.25
4620,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75,8.75


In [70]:
df1.describe()

,Unnamed: 0,order_id,quantity,item_price,item_price_unit
count,4622.000000,4622.000000,4622.000000,4622.000000,4622.000000
mean,2310.500000,927.254868,1.075725,7.464336,7.084424
std,1334.400802,528.890796,0.410186,4.245557,3.665487
min,0.000000,1.000000,1.000000,1.090000,1.090000
25%,1155.250000,477.250000,1.000000,3.390000,2.950000
50%,2310.500000,926.000000,1.000000,8.750000,8.750000
75%,3465.750000,1393.000000,1.000000,9.250000,9.250000
max,4621.000000,1834.000000,15.000000,44.250000,11.890000


In [71]:
list(df1)

['Unnamed: 0',
 'order_id',
 'quantity',
 'item_name',
 'choice_description',
 'item_price',
 'item_price_unit']

Cette fois-ci, on peut obtenir une description statistique des prix que l'on utilisera + tard.

#### Statistiques sur les articles commandés

In [72]:
# Nombre total d'items commandés
df1.groupby(['item_name']).sum().sort_values(by='quantity', ascending=False).quantity.sum()

4972

In [73]:
""" !!! Ne prends pas en compte les quantités """
# Nombre de commandes de chacun des articles
df1["item_name"].value_counts()

Chicken Bowl                             726
Chicken Burrito                          553
Chips and Guacamole                      479
Steak Burrito                            368
Canned Soft Drink                        301
Steak Bowl                               211
Chips                                    211
Bottled Water                            162
Chicken Soft Tacos                       115
Chips and Fresh Tomato Salsa             110
Chicken Salad Bowl                       110
Canned Soda                              104
Side of Chips                            101
Veggie Burrito                            95
Barbacoa Burrito                          91
Veggie Bowl                               85
Carnitas Bowl                             68
Barbacoa Bowl                             66
Carnitas Burrito                          59
Steak Soft Tacos                          55
6 Pack Soft Drink                         54
Chips and Tomatillo Red Chili Salsa       48
Chicken Cr

On peut voir quelques difficultés propres aux variables qualitatives. Certaines valeurs représentent la même chose mais ont une écriture différente.

Par exemple _Chips and Tomatillo-Green Chili Salsa_ et _Chips and Tomatillo Green Chili Salsa_ devraient désigner le même article, mais dans notre jeu de données ce sont 2 valeurs différentes à cause du tiret dans la chaîne de caractères.\
On pourrait aller plus loin en disant que _Chips and Tomatillo_ est l'information principale et que la sauce utilisée pourrait être dans la description, surtout si les prix sont les même quelle que soit la sauce utilisée.


In [74]:
# On ne garde que la valeur "Chips and Tomatillo"
df1["item_name"] = df1["item_name"].replace(regex=r"Chips and Tomatillo.*", value="Chips and Tomatillo")

In [75]:
# Prise en compte des quantités
df1.groupby(['item_name']).sum().sort_values(by='quantity', ascending=False).quantity

item_name
Chicken Bowl                          761
Chicken Burrito                       591
Chips and Guacamole                   506
Steak Burrito                         386
Canned Soft Drink                     351
Chips                                 230
Steak Bowl                            221
Bottled Water                         211
Chips and Tomatillo                   153
Chips and Fresh Tomato Salsa          130
Canned Soda                           126
Chicken Salad Bowl                    123
Chicken Soft Tacos                    120
Side of Chips                         110
Veggie Burrito                         97
Barbacoa Burrito                       91
Veggie Bowl                            87
Carnitas Bowl                          71
Barbacoa Bowl                          66
Carnitas Burrito                       60
Steak Soft Tacos                       56
6 Pack Soft Drink                      55
Chicken Crispy Tacos                   50
Carnitas Soft Tacos     

In [90]:
# On cherche les compositions d'ingrédients les + courantes dans une Chicken Bowl
# Ne prends pas en compte les quantité
df1[df1['item_name'] == "Chicken Bowl"]["choice_description"].value_counts().iloc[0:20]

[Fresh Tomato Salsa, [Fajita Vegetables, Rice]]                                              17
[Fresh Tomato Salsa, [Rice, Black Beans, Cheese, Sour Cream, Lettuce]]                       14
[Fresh Tomato Salsa, [Rice, Black Beans, Cheese, Sour Cream, Guacamole, Lettuce]]            14
[Fresh Tomato Salsa, [Rice, Cheese, Sour Cream, Lettuce]]                                    13
[Fresh Tomato Salsa, [Rice, Black Beans, Cheese]]                                            13
[Fresh Tomato Salsa, [Rice, Cheese, Lettuce]]                                                13
[Tomatillo Green Chili Salsa, [Rice, Pinto Beans, Sour Cream, Lettuce]]                      12
[Fresh Tomato Salsa, [Rice, Black Beans, Cheese, Sour Cream]]                                11
[Fresh Tomato Salsa, [Rice, Cheese, Sour Cream]]                                             10
[Roasted Chili Corn Salsa, [Rice, Black Beans, Cheese, Sour Cream, Lettuce]]                 10
[Fresh Tomato Salsa, [Fajita Vegetables,

In [39]:
import ast
# initializing string representation of a list
ini_list = "[Fresh Tomato Salsa, [Fajita Vegetables, Rice]]"

# Converting string to list
res = ast.literal_eval(ini_list)


# printing final result and its type
print(res)
print(type(res))

['Fresh Tomato Salsa', ['Fajita Vegetables', 'Rice']]
<class 'list'>


In [40]:
help(ast.literal_eval)

Help on function literal_eval in module ast:

literal_eval(node_or_string)
    Safely evaluate an expression node or a string containing a Python
    expression.  The string or node provided may only consist of the following
    Python literal structures: strings, bytes, numbers, tuples, lists, dicts,
    sets, booleans, and None.



In [34]:
df1[df1["item_name"] == "Chicken Bowl"]

,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price,item_price_unit
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,8.49
5,5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98,10.98
13,13,7,1,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",11.25,11.25
19,19,10,1,Chicken Bowl,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",8.75,8.75
26,26,13,1,Chicken Bowl,"[Roasted Chili Corn Salsa (Medium), [Pinto Bea...",8.49,8.49
...,...,...,...,...,...,...,...
4590,4590,1825,1,Chicken Bowl,"[Roasted Chili Corn Salsa, [Rice, Black Beans,...",11.25,11.25
4591,4591,1825,1,Chicken Bowl,"[Tomatillo Red Chili Salsa, [Rice, Black Beans...",8.75,8.75
4595,4595,1826,1,Chicken Bowl,"[Tomatillo Green Chili Salsa, [Rice, Black Bea...",8.75,8.75
4599,4599,1827,1,Chicken Bowl,"[Roasted Chili Corn Salsa, [Cheese, Lettuce]]",8.75,8.75


In [13]:
# On cherche tous les articles contenant du guacamole
# Le guacamole se trouve parfois dans le nom de l'article, parfois dans sa description
df1[(df1["item_name"].str.contains("Guacamole")) | (df1["choice_description"].str.contains("Guacamole"))]

,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price
5,5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98
7,7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75
10,10,5,1,Chips and Guacamole,NaN,4.45
13,13,7,1,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",11.25
14,14,7,1,Chips and Guacamole,NaN,4.45
...,...,...,...,...,...,...
4611,4611,1830,1,Veggie Burrito,"[Tomatillo Green Chili Salsa, [Rice, Fajita Ve...",11.25
4616,4616,1832,1,Chips and Guacamole,NaN,4.45
4617,4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75
4618,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75


#### Statistiques sur les prix

In [14]:
# Chiffre d'affaire (en dollars)
df1["item_price"].sum()

34500.16

In [42]:
df1

,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price,item_price_unit
0,0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,2.39
1,1,1,1,Izze,[Clementine],3.39,3.39
2,2,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,3,1,1,Chips and Tomatillo,NaN,2.39,2.39
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,8.49
...,...,...,...,...,...,...,...
4617,4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75,11.75
4618,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75,11.75
4619,4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25,11.25
4620,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75,8.75


In [48]:
df1.groupby(['order_id']).min()[df1.groupby(['order_id']).min().quantity > 1]

C:\Users\julie\AppData\Local\Temp\ipykernel_16336\491491132.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  df1.groupby(['order_id']).min()[df1.groupby(['order_id']).min().quantity > 1]
C:\Users\julie\AppData\Local\Temp\ipykernel_16336\491491132.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  df1.groupby(['order_id']).min()[df1.groupby(['order_id']).min().quantity > 1]


,Unnamed: 0,quantity,item_name,item_price,item_price_unit
order_id,,,,,
2,4,2,Chicken Bowl,16.98,8.49
60,135,2,Chicken Salad Bowl,22.50,11.25
94,213,2,Chicken Salad Bowl,22.50,11.25
105,241,2,Chicken Burrito,17.50,8.75
123,281,2,Steak Salad Bowl,23.78,11.89
...,...,...,...,...,...
1593,3976,2,Chicken Bowl,16.98,8.49
1634,4082,2,Steak Bowl,18.50,9.25
1640,4093,2,Chicken Burrito,17.50,8.75


In [15]:
# Faire des statistiques des prix sur chacune des commandes
# Il faut d'abord grouper les prix par commande, puis afficher leur description statistique
df1.groupby(['order_id']).sum()

,Unnamed: 0,quantity,item_price
order_id,,,
1,6,4,11.56
2,4,2,16.98
3,11,2,12.67
4,15,2,21.00
5,19,2,13.70
...,...,...,...
1830,9221,2,23.00
1831,13839,3,12.90
1832,9231,2,13.20


In [55]:
# Chiffre d'affaire par plat
df1.groupby(['item_name']).sum().describe()

,Unnamed: 0,order_id,quantity,item_price,item_price_unit
count,4.700000e+01,47.000000,47.000000,47.000000,47.000000
mean,2.272156e+05,91186.638298,105.787234,734.045957,696.685319
std,3.645126e+05,146250.491940,163.465052,1422.413420,1352.203875
min,6.740000e+02,279.000000,1.000000,3.000000,3.000000
25%,1.743700e+04,6979.000000,8.500000,67.825000,65.905000
50%,9.376600e+04,37673.000000,40.000000,302.560000,250.460000
75%,2.279500e+05,91582.000000,121.500000,635.095000,630.470000
max,1.779909e+06,713926.000000,761.000000,7342.730000,7011.510000


In [16]:
df1.groupby(['order_id']).sum().describe()

,Unnamed: 0,quantity,item_price
count,1834.000000,1834.000000,1834.000000
mean,5822.863141,2.711014,18.811429
std,4675.730261,1.677624,11.652512
min,4.000000,1.000000,10.080000
25%,2464.000000,2.000000,12.572500
50%,5235.000000,2.000000,16.200000
75%,8022.000000,3.000000,21.960000
max,53245.000000,35.000000,205.250000


En moyenne, une commande a coûté 18.81$

#### Conclusion

TODO: À revoir, à corriger

Les variables qualitatives sont parfois piégeuses et nécessitent des traitements de données pour pouvoir tirer une analyse non-biaisée de ces variables.

Par ailleurs, il faut toujours faire attention au type des colonnes de notre dataset et comprendre la forme des valeurs non-atomiques (ici _choice\_description_ qui est une liste qui peut contenir une autre liste) pour savoir les traiter ou savoir ce qu'on perd dans notre analyse en les ignorant.